In [1]:


# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

#for saving xml
import xml.etree.cElementTree as ET
from random import randrange
import os

In [6]:
os.chdir("./test_images")

In [137]:
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)

In [138]:
def load_and_resize_image(url, orig_size=True , new_width=256, new_height=256,
                              display=False):
  filename=None
  ###for downloading image from url
  #_, filename = tempfile.mkstemp(suffix=".jpg")
  #response = urlopen(url)
  #image_data = response.read()
  #image_data = BytesIO(image_data)
  
  
  pil_image = Image.open(url)
  if orig_size==True:
    new_width, new_height=pil_image.size
    
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  if filename is None:
     filename=((url.split('/'))[-1])
  pil_image_rgb.save(filename)
  #print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return (filename,pil_image_rgb )

In [139]:
def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=2,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin




In [140]:
def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

In [35]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" 
#module_handle="https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"
detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [141]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [142]:
def generate_xml(image ,file_name_ext , class_names , boxes , class_scores):
    im_width , im_height= image.size
    annotation = ET.Element("annotation")
    ET.SubElement(annotation, "filename").text = file_name_ext
    size = ET.SubElement(annotation, "size")
    ET.SubElement(size, "width").text = str(im_width)
    ET.SubElement(size, "height").text = str(im_height)
    ET.SubElement(size, "depth").text = "3"
    
    
    for j,i in enumerate(boxes):
        if class_scores[j]>0.5:
            ymin, xmin, ymax, xmax = tuple(boxes[j])
            objectBox = ET.SubElement(annotation, "object")
            ET.SubElement(objectBox, "name").text = class_names[j].decode("ascii")
            ET.SubElement(objectBox, "score").text = str(class_scores[j]*100)
            ET.SubElement(objectBox, "pose").text = "Unspecified"
            ET.SubElement(objectBox, "truncated").text = "0"
            ET.SubElement(objectBox, "difficult").text = "0"
            bndBox = ET.SubElement(objectBox, "bndbox")
            ET.SubElement(bndBox, "xmin").text = str(xmin*im_width)
            ET.SubElement(bndBox, "ymin").text = str(ymin*im_height)
            ET.SubElement(bndBox, "xmax").text = str(xmax*im_width)
            ET.SubElement(bndBox, "ymax").text = str(ymax*im_height)
    
    file_name,ext = file_name_ext.split('.') 
    print(file_name)
    ref = ET.ElementTree(annotation)
    ref.write(file_name + ".xml")

In [143]:
def run_detector(detector, path):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  #return converted_img.shape
  
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

  print("Found %d objects." % len(result["detection_scores"]))
  print("Inference time: ", end_time-start_time)

  #image_with_boxes = draw_boxes(
  #    img.numpy(), result["detection_boxes"],
  #    result["detection_class_entities"], result["detection_scores"])

  #generate_xml(img ,path , result["detection_class_entities"] , result["detection_boxes"] , result["detection_scores"])
  
  return result
  #display_image(image_with_boxes)

In [144]:
#for loading single image

url='img.jpg'
image_path, img = load_and_resize_image(url,orig_size=1, new_width=256, new_height=256,display=False)
result=run_detector(detector, image_path)


Found 100 objects.
Inference time:  49.91003608703613


In [145]:
img.size

(1156, 781)

In [146]:
f=generate_xml(img ,image_path , result["detection_class_entities"] , result["detection_boxes"] , result["detection_scores"])

img


In [132]:
result["detection_scores"]>0.5]

array([ True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [80]:
print(f)

In [81]:
root = f.getroot()


In [82]:
root

<Element 'annotation' at 0x00000291C2AE89F8>

In [ ]:
for image_url in image_urls:
  start_time = time.time()
  image_path = load_and_resize_image(image_url, 640, 480)
  run_detector(detector, image_path)
  end_time = time.time()
  print("Inference time:",start_time-end_time)

In [14]:
for i,j in enumerate([1,3]):
    print(i)

0
1


In [109]:
a=np.array(np.random.randn(3,3))

In [110]:
a

array([[ 1.83169962, -0.94188599, -0.19880135],
       [ 0.42771464,  0.66774732, -0.4164674 ],
       [ 0.45631514,  0.35308784, -0.16131391]])

In [112]:
(m,n)=np.nonzero(a > 0)

In [113]:
m

array([0, 1, 1, 2, 2], dtype=int64)

In [114]:
n

array([0, 0, 1, 0, 1], dtype=int64)

In [115]:
m.shape

(5,)

In [116]:
type(m)

numpy.ndarray

In [125]:
s=np.c_[m,n]

In [126]:
s

array([[0, 0],
       [1, 0],
       [1, 1],
       [2, 0],
       [2, 1]], dtype=int64)

In [127]:
a[s]

array([[[ 1.83169962, -0.94188599, -0.19880135],
        [ 1.83169962, -0.94188599, -0.19880135]],

       [[ 0.42771464,  0.66774732, -0.4164674 ],
        [ 1.83169962, -0.94188599, -0.19880135]],

       [[ 0.42771464,  0.66774732, -0.4164674 ],
        [ 0.42771464,  0.66774732, -0.4164674 ]],

       [[ 0.45631514,  0.35308784, -0.16131391],
        [ 1.83169962, -0.94188599, -0.19880135]],

       [[ 0.45631514,  0.35308784, -0.16131391],
        [ 0.42771464,  0.66774732, -0.4164674 ]]])

In [128]:
(a[s]).shape

(5, 2, 3)